In [131]:
"""
Solution 1: use traditional machine learning model
"""

import os
import sys
import numpy as np
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import random

#define captcha class for train, predict
class Captcha(object):
    def __init__(self, is_train = False, model_pth = None):
        #function mode control and moel saved path
        self.model_pth = model_pth
        self.model = None

        if not is_train:
            try:
                with open(self.model_pth, "rb") as ofl:
                    self.model = pickle.load(ofl)
                    print(f"*** model loading success -> {self.model_pth}")
            except:
                raise Exception(f" *** model not found -> {self.model_pth}")
                
    def fit(self, train_sample, train_label):
        print(f"***** Start training using {len(train_label)} samples")
        self.model = LogisticRegression(penalty = "l1", solver = "saga", random_state = 20240116)
        self.model.fit(train_sample, train_label)
        if self.model_pth:
            with open(self.model_pth, "wb") as ofl:
                pickle.dump(self.model, ofl)
                print(f"*** model saved to {self.model_pth}")
        print(f"*** train done")

    def __call__(self, im_path, save_path):
        """
        Algo for inference
        args:
            im_path: image file to load and to infer, 
                
            save_path: output file path to save the one-line outcome
        """
        if im_path.endswith(".txt"):
            #load text format image data same sample
            input_data, _ = load_captchas_image_label(im_path)
        else:
            #load image data
            img = cv2.imread(im_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            input_data = img.mean(axis = 2).reshape(-1,)
            
        pred = self.model.predict([input_data])[0]

        if save_path:
            nm = im_path.split("/")[-1].split(".")[0] + ".txt"
            o_file = os.path.join(save_path, nm)
            with open(o_file, "wt") as ofl:
                ofl.write(pred)

        return pred
        

def load_captchas_image_label(filename_input, filename_input_label = None):
    #load samples
    pixels = []
    with open(filename_input, "r") as f:
        next(f)
        for line in f:
            for item in line.split(" "):
                vals = [int(val) for val in item.split(",")]
                pixels.append(sum(vals)/len(vals))
    #
    label = None
    if filename_input_label:
        with open(filename_input_label, "r") as f:
            label = f.read().strip()
    
    return pixels, label
        
# Step-1: Prepare data for model training & test
captcha_image_folder = "./input"
captcha_image_label_folder = "./output"
# inputs = []
# labels = []
x_train, x_test, y_train, y_test, id_train, id_test = [], [], [], [], [], []
ratio_train = 0.8

random.seed(20240116)
for filename in os.listdir(captcha_folder):
    if filename.endswith(".txt"):
        input_image_fl = os.path.join(captcha_folder, filename)
        input_image_label_fl = os.path.join(captcha_image_label_folder, "output" + filename.split('input')[1])
        isample = load_captchas_image_label(input_image_fl, input_image_label_fl)
        
        if random.random() <= ratio_train:   
            x_train.append(isample[0])
            y_train.append(isample[1])
            id_train.append(filename)
        else:
            x_test.append(isample[0])
            y_test.append(isample[1])
            id_test.append(filename)
            
# Split the data into training and testing sets
# x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

#Step-2:
inst_learner = Captcha(is_train = True, model_pth = "./model/captcha_lr.pkl")
inst_learner.fit(x_train, y_train)

***** Start training using 22 samples
*** model saved to ./model/captcha_lr.pkl
*** train done


/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [132]:
id_test

['input10.txt', 'input22.txt', 'input23.txt']

In [136]:
inst_learner = Captcha(is_train = False, model_pth = "./model/captcha_lr.pkl")

#test inference using text
for v in id_test:
    name = os.path.join(captcha_image_folder, v) 
    print(name)
    inst_learner(name, "./output_lr")


*** model loading success -> ./model/captcha_lr.pkl
./input/input10.txt
./input/input22.txt
./input/input23.txt


In [142]:
"""
Solution 2: use SOTA Image-to-Text pretrained model
"""

from transformers import pipeline
from PIL import Image

class CaptchaI2T(object):
    def __init__(self, ):
        self.pipe = pipeline("image-to-text", model="microsoft/trocr-large-printed")

    def __call__(self, im_path, save_path):
        """
        Algo for inference
        args:
            im_path: image file to load and to infer, 
                
            save_path: output file path to save the one-line outcome
        """
        try:
            input_data = Image.open(im_path).convert("RGB")
            input_data.show()
            pred2 = self.pipe(input_data)[0]
            pred = pred2["generated_text"]
            # print(pred)
            
            if save_path:
                nm = im_path.split("/")[-1].split(".")[0] + ".txt"
                o_file = os.path.join(save_path, nm)
                print(o_file)
                with open(o_file, "wt") as ofl:
                    ofl.write(pred)
        except:
            print(f"*** Inference failed -> {im_path}")
            pred = None

        return pred

#test
inst_i2t = CaptchaI2T()

#test inference using text
for v in id_test:
    name = os.path.join(captcha_image_folder, v.split(".")[0] + ".jpg") 
    print(name)
    inst_i2t(name, "./output_i2t")


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


./input/input10.jpg


/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


./output_i2t/input10.txt
./input/input22.jpg


/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


./output_i2t/input22.txt
./input/input23.jpg


/home/gs/miniconda3/envs/t2v/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


./output_i2t/input23.txt


In [151]:
from diffusers.utils import make_image_grid
img_lst = []
pred_lr = []
pred_i2t = []
label = []

for v in id_test:
    name = os.path.join("./input", v.split(".")[0] + ".jpg") 
    img_lst.append(Image.open(name))
    name = os.path.join("./output", "output"+v.split("input")[1])
    with open(name, "rt") as ofl:
        s = ofl.read().strip("\n")
        label.append(s)

    name = os.path.join("./output", "output"+v.split("input")[1])
    with open(name, "rt") as ofl:
        s = ofl.read().strip("\n")
        label.append(s)

    name = os.path.join("./output_lr", "input"+v.split("input")[1])
    with open(name, "rt") as ofl:
        s = ofl.read().strip("\n")
        pred_lr.append(s)

    name = os.path.join("./output_i2t", "input"+v.split("input")[1])
    with open(name, "rt") as ofl:
        s = ofl.read().strip("\n")
        pred_i2t.append(s)

    


In [154]:
print("Input image")
make_image_grid(img_lst, 1, 3)

Input image


In [155]:
print("Model prediction: Logistic regression")
print(pred_lr)

print("Model prediction: image-to-text")
print(pred_i2t)


Model prediction: Logistic regression
['ZRMQU', '6O5W1', 'WGST7']
Model prediction: image-to-text
['GZMBA', 'HCE91', 'WELXV']


In [ ]:
"""
From above 3 samples, image-to-text is quite better than traditional classifier such as logic regression
"""